In [8]:
import os
import threading
import tkinter as tk
import speech_recognition as sr
import pyttsx3
import pygame
from googletrans import Translator
from gtts import gTTS
from datetime import date
import pandas as pd
import random
import language_tool_python
from functools import partial

# Initialize necessary components
voice = pyttsx3.init()
voice_command = pyttsx3.init()
translator = Translator()
my_tool = language_tool_python.LanguageTool('en-US')

# Load data from Excel into a dictionary of questions
df = pd.read_excel('questions.xlsx')
categories = {
    'family': list(df['My Family & Friends'].dropna()),
    'hobbies': list(df['My Hobbies & Interests'].dropna()),
    'neighbour': list(df['My Neighbourhood'].dropna()),
    'food': list(df['Food & Nutrition'].dropna()),
    'school': list(df['My School'].dropna())
}

language_mapping = {
    'afrikaans': 'af', 'albanian': 'sq', 'amharic': 'am', 'arabic': 'ar',
    'armenian': 'hy', 'azerbaijani': 'az', 'basque': 'eu', 'belarusian': 'be',
    'bengali': 'bn', 'bosnian': 'bs', 'bulgarian': 'bg', 'catalan': 'ca',
    'cebuano': 'ceb', 'chichewa': 'ny', 'chinese (simplified)': 'zh-cn',
    'chinese (traditional)': 'zh-tw', 'corsican': 'co', 'croatian': 'hr',
    'czech': 'cs', 'danish': 'da', 'dutch': 'nl', 'english': 'en', 'esperanto': 'eo',
    'estonian': 'et', 'filipino': 'tl', 'finnish': 'fi', 'french': 'fr', 'frisian': 'fy',
    'galician': 'gl', 'georgian': 'ka', 'german': 'de', 'greek': 'el', 'gujarati': 'gu',
    'haitian creole': 'ht', 'hausa': 'ha', 'hawaiian': 'haw', 'hebrew': 'he', 'hindi': 'hi',
    'hmong': 'hmn', 'hungarian': 'hu', 'icelandic': 'is', 'igbo': 'ig', 'indonesian': 'id',
    'irish': 'ga', 'italian': 'it', 'japanese': 'ja', 'javanese': 'jw', 'kannada': 'kn',
    'kazakh': 'kk', 'khmer': 'km', 'korean': 'ko', 'kurdish (kurmanji)': 'ku', 'kyrgyz': 'ky',
    'lao': 'lo', 'latin': 'la', 'latvian': 'lv', 'lithuanian': 'lt', 'luxembourgish': 'lb',
    'macedonian': 'mk', 'malagasy': 'mg', 'malay': 'ms', 'malayalam': 'ml', 'maltese': 'mt',
    'maori': 'mi', 'marathi': 'mr', 'mongolian': 'mn', 'myanmar (burmese)': 'my', 'nepali': 'ne',
    'norwegian': 'no', 'odia': 'or', 'pashto': 'ps', 'persian': 'fa', 'polish': 'pl', 'portuguese': 'pt',
    'punjabi': 'pa', 'romanian': 'ro', 'russian': 'ru', 'samoan': 'sm', 'scots gaelic': 'gd',
    'serbian': 'sr', 'sesotho': 'st', 'shona': 'sn', 'sindhi': 'sd', 'sinhala': 'si', 'slovak': 'sk',
    'slovenian': 'sl', 'somali': 'so', 'spanish': 'es', 'sundanese': 'su', 'swahili': 'sw',
    'swedish': 'sv', 'tajik': 'tg', 'tamil': 'ta', 'telugu': 'te', 'thai': 'th', 'turkish': 'tr',
    'ukrainian': 'uk', 'urdu': 'ur', 'uyghur': 'ug', 'uzbek': 'uz', 'vietnamese': 'vi', 'welsh': 'cy',
    'xhosa': 'xh', 'yiddish': 'yi', 'yoruba': 'yo', 'zulu': 'zu'
}
def takecommand(label, message):
    r = sr.Recognizer()
    with sr.Microphone() as source:
        label.config(text=message)
        r.pause_threshold = 1
        r.adjust_for_ambient_noise(source, duration=1)  # Adjust for ambient noise
        audio = r.listen(source)

    try:
        label.config(text="Recognizing...")
        query = r.recognize_google(audio, language='en-in')
        root.after(0, lambda: label.config(text=f"User said: {query}"))
        return query
    except Exception as e:
        root.after(0, lambda: label.config(text="Say that again, please."))
        print(f"Error in speech recognition: {e}")
        return "None"

def destination_language(label):
    message = "Please tell me the language to which you want to translate: \nExample: Hindi, English, etc."
    voice_command.say(message)
    voice_command.runAndWait()
    to_lang = takecommand(label, message)

    while to_lang == "None":
        to_lang = takecommand(label, message)

    to_lang = to_lang.lower()
    return to_lang

def translatorx(label):
    try:
        # Get the destination language from the user
        to_lang = destination_language(label)

        # Prompt the user to provide the sentence to translate
        message = "Please say the sentence you want to translate."
        voice_command.say(message)
        voice_command.runAndWait()

        query = takecommand(label, message)

        while query == "None":
            query = takecommand(label, message)

        if to_lang in language_mapping:
            to_lang = language_mapping[to_lang]

        translation = translator.translate(query, dest=to_lang)
        translated_text = translation.text

        # Update the label with the translated text in a thread-safe manner
        root.after(0, lambda: label.config(text=f"Translated text: {translated_text}"))

        # Play the translated text with gTTS and pygame
        tts = gTTS(text=translated_text, lang=to_lang, slow=False)
        file_path = 'sound.mp3'
        tts.save(file_path)

        pygame.init()
        pygame.mixer.music.load(file_path)
        pygame.mixer.music.play()

        # Wait for the sound to finish playing
        while pygame.mixer.music.get_busy():
            pygame.time.Clock().tick(10)

        # Clean up resources
        pygame.quit()
        os.remove(file_path)
    except Exception as e:
        root.after(0, lambda: label.config(text="Error in translation."))
        print("Error:", e)

def destination_language_thread(label):
    thread = threading.Thread(target=destination_language, args=(label,))
    thread.start()


def translator_thread(label):
    thread = threading.Thread(target=translatorx, args=(label,))
    thread.start()


def grammar_checker(user_text, label):
    if user_text.lower() != "none":
        matches = my_tool.check(user_text)

        if not matches:
            corrected_text = "Amazing! Your grammar was correct!"
        else:
            corrected_text = my_tool.correct(user_text)

            # Provide detailed feedback on errors and corrections
            mistake_info = "\n".join(
                f"Mistake: {user_text[m.offset:m.offset + m.errorLength]} | Correction: {m.replacements[0]}"
                for m in matches if m.replacements
            )

            corrected_text = f"Original: {user_text}\nCorrected: {corrected_text}\nDetails:\n{mistake_info}"

        # Update the label in a thread-safe manner
        root.after(0, lambda: label.config(text=corrected_text))
    else:
        root.after(0, lambda: label.config(text="Invalid input for grammar check."))

def speak_question(question):
    voice.say(question)
    voice.runAndWait()

def ask_question(category, label):
    global question
    question = random.choice(categories[category])
    root.after(0, lambda: label.config(text=question))

    # Speak the question and listen for the response
    threading.Thread(target=speak_question, args=(question,)).start()
    threading.Thread(target=capture_response, args=(label,)).start()

def capture_response(label):
    message = question
    response = takecommand(label, message)
    response = response[0].upper()

    if response != "None":
        threading.Thread(target=grammar_checker, args=(response, label)).start()
    else:
        root.after(0, lambda: label.config(text="No valid response. Please try again."))

# GUI setup for Tkinter
root = tk.Tk()
root.geometry("1000x1000")
root.title("ECHO")

#Make a Canvas
canvas1 = tk.Canvas(root, width = 1000, height = 1000, bg='blue')
canvas1.pack(fill=tk.BOTH, expand=True)
# Load the image
bg_image = tk.PhotoImage(file="bg.png")
# Add the image to the canvas
canvas1.create_image(0, 0, anchor=tk.NW, image=bg_image)

text_title = canvas1.create_text(650, 70, anchor = "nw")
canvas1.itemconfig(text_title, text="ECHO", font=('Times New Roman', 80, 'bold'))

text_subtitle = canvas1.create_text(550, 190, anchor = "nw")
canvas1.itemconfig(text_subtitle, text="Empowering Conversational Hub & Outreach", font=('Times New Roman', 20, 'italic'))

text_date = canvas1.create_text(1400, 5, anchor = "nw")
canvas1.itemconfig(text_date, text=date.today().strftime("%B %d, %Y"), font=('Arial', 14, 'italic'))

hor_up = canvas1.create_rectangle(500, 60, 790, 65, fill = "#8DBDAD", outline="#C9E6C0")
canvas1.pack(side = "top", fill = "both", expand = True)
ver_left = canvas1.create_rectangle(500, 60, 505, 255, fill = "#8DBDAD", outline="#C9E6C0")
canvas1.pack(side = "top", fill = "both", expand = True)
hor_down = canvas1.create_rectangle(840, 250, 1100, 255, fill = "#8DBDAD", outline="#C9E6C0")
canvas1.pack(side = "top", fill = "both", expand = True)
ver_right = canvas1.create_rectangle(1100, 60, 1105, 255, fill = "#8DBDAD", outline="#C9E6C0")
canvas1.pack(side = "top", fill = "both", expand = True)

text_intro = canvas1.create_text(800, 430)
canvas1.itemconfig(text_intro, text='How do you want to practice today?', font=('Montserrat', 22))

# Define the function to set up the translation section
def setup_translation():
    canvas1.delete(text_title)
    canvas1.delete(text_subtitle)
    canvas1.delete(text_intro)
    canvas1.delete(hor_up)
    canvas1.delete(hor_down)
    canvas1.delete(ver_left)
    canvas1.delete(ver_right)
    con_button.destroy()
    trans_button.destroy()

    smalltitle = tk.Label(root, text='ECHO', fg='#8DBDAD', font=('Times New Roman', 12, 'bold' , 'underline'))
    smalltitle.pack()
    canvas1.create_window(815, 18, window=smalltitle)

    trans_title_label = tk.Label(root, text='Translation', font=('Eras Demi ITC', 23))
    canvas1.create_window(815, 65, window=trans_title_label)
    

    div_line = canvas1.create_rectangle(0, 0, 2000, 5, fill = "#8DBDAD", outline="#C9E6C0")
    canvas1.pack(side = "top", fill = "both", expand = True)

    # Label to display translated text
    label_con_ans = tk.Label(root, text='', bg='#8DBDAD', font=('Montserrat', 20), fg='white', wraplength=500)
    canvas1.create_window(800, 400, window=label_con_ans)

    # Button to start the translation process
    trans_intro_btn = tk.Button(
        root,
        text='I am ready. Let\'s go!',
        command=lambda: translator_thread(label_con_ans),
        font=('Montserrat', 18)
    )
    canvas1.create_window(815, 200, window=trans_intro_btn)

# Function to set up the conversation section
def setup_conversation_gui():
    canvas1.delete(text_title)
    canvas1.delete(text_subtitle)
    canvas1.delete(text_intro)
    canvas1.delete(hor_up)
    canvas1.delete(hor_down)
    canvas1.delete(ver_left)
    canvas1.delete(ver_right)
    con_button.destroy()
    trans_button.destroy()

    smalltitle = tk.Label(root, text='ECHO', fg='#8DBDAD', font=('Times New Roman', 12, 'bold' , 'underline'))
    smalltitle.pack()
    canvas1.create_window(815, 18, window=smalltitle)

    con_title_label = tk.Label(root, text='Conversation', font=('Eras Demi ITC', 23))
    canvas1.create_window(815, 65, window=con_title_label)
    
    div_line = canvas1.create_rectangle(0, 0, 2000, 5, fill = "#8DBDAD", outline="#C9E6C0")
    canvas1.pack(side = "top", fill = "both", expand = True)
    #div_line2 = canvas1.create_rectangle(0, 90, 2000, 95, fill = "#8DBDAD", outline="#C9E6C0")
    #canvas1.pack(side = "top", fill = "both", expand = True)
    
    label_con_ans = tk.Label(root, text='Choose one topic...', bg='#8DBDAD', font=('Montserrat', 14), fg='white')
    canvas1.create_window(400, 140, window=label_con_ans)

    # Label to display questions and responses
    label = tk.Label(root, text='', fg='white', font=('Montserrat', 20), bg='#8DBDAD', wraplength=800)
    canvas1.create_window(800, 560, window=label)

    # Create a frame to hold category buttons
    button_frame = tk.Frame(root)
    canvas1.create_window(800, 350, window=button_frame)

    # Create buttons for each category to start a conversation
    category_buttons = {
        'family': partial(ask_question, 'family', label),
        'hobbies': partial(ask_question, 'hobbies', label),
        'neighbour': partial(ask_question, 'neighbour', label),
        'school': partial(ask_question, 'school', label),
        'food': partial(ask_question, 'food', label),
    }

    button_texts = ["My Family & Friends", "My Hobbies & Interests", "My Neighbourhood", "My School", "Food & Nutrition"]

    # Add buttons for conversation categories
    for i, (category, cmd) in enumerate(category_buttons.items()):
        button = tk.Button(
            button_frame,
            text=button_texts[i],
            command=cmd,
            font=('Eras Demi ITC', 14)
        )
        button.pack(side=tk.LEFT, padx=30, pady=40)

# Main menu with options to choose Translation or Conversation
trans_button = tk.Button(
    root,
    text='TRANSLATION',
    command=setup_translation,
    height=2,
    width=14,
    font=('Eras Demi ITC', 17)
)
canvas1.create_window(600, 530, window=trans_button)

con_button = tk.Button(
    root,
    text='CONVERSATION',
    command=setup_conversation_gui,
    height=2,
    width=14,
    font=('Eras Demi ITC', 17)
)
canvas1.create_window(1000, 530, window=con_button)

# Start the Tkinter main loop
root.mainloop()